- 발표용으로 순서가 변경된 파일, 원본은  SB_NA_원본.ipynb

# ★★★★발표할 내용★★★★

## 모든 의원에 대한 뉴스 데이터 수집

In [149]:
MPdict = {} # 저장용으로 빈 딕셔너리 생성 
nMP = len(MPnames) # 국회의원수
for i, name in enumerate(MPnames): # 국회의원 이름 리스트로 for문 실행
    print(nMP - i, end=" ") # run하고 어디가 실행되고 있는지 확인하고 싶어서 추가

    mp = MPdata(name) # 딕셔너리에 키값으로 저장할 MPdata의 인스턴스 생성
    
    df = getNewsAbout(name) # name으로 검색한 뉴스데이터 크롤링 (기사 제목과 요약을 포함)
    mp.save_org(df) # 전처리 전 데이터 원본을 mp에 저장
    
    df_r1 = reduction1(df, name) # 1차 전처리 : 중복제거, 한글만 남기기, 검색어 삭제
    mp.save_r1(df_r1) # 1차 전처리 후 데이터를 mp에 저장
    
    text = reduction2(df_r1) # 2차 전처리 : 모든 한글을 하나의 str으로 합친다.
    mp.save_text(text) # 2차 전처리 후 데이터를 mp에 저장
    
    MPdict[name] = mp # 저장 형식 {name : mp(MPdata object), ...}

299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 250 249 248 247 246 245 244 243 242 241 240 239 238 237 236 235 234 233 232 231 230 229 228 227 226 225 224 223 222 221 220 219 218 217 216 215 214 213 212 211 210 209 208 207 206 205 204 203 202 201 200 199 198 197 196 195 194 193 192 191 190 189 188 187 186 185 184 183 182 181 180 179 178 177 176 175 174 173 172 171 170 169 168 167 166 165 164 163 162 161 160 159 158 157 156 155 154 153 152 151 150 149 148 147 146 145 144 143 142 141 140 139 138 137 136 135 134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33

In [202]:
type(MPdict), len(MPdict)

(dict, 297)

- 아까 확인했던 동명이인이 덮어 씌워짐.
- 김병욱(33, 34), 이수진(191, 192)

In [136]:
MP[MP.duplicated(subset=['의원명'], keep='last')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
33,김병욱,더불어민주당,[국토교통위원회],경기 성남시분당구을,남,재선,지역구
191,이수진,더불어민주당,"[기획재정위원회, 예산결산특...",서울 동작구을,여,초선,지역구


In [137]:
MP[MP.duplicated(subset=['의원명'], keep='first')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
34,김병욱,국민의힘,"[교육위원회, 예산결산특별위원회]",경북 포항시남구울릉군,남,초선,지역구
192,이수진,더불어민주당,"[민생경제안정특별위원회, 국...",비례대표,여,초선,비례대표


## 파일로 저장

In [203]:
dt = datetime.now()
dt = dt.strftime('_%Y%m%d_%H%M%S')
with open(f'MP_news_data_{dt}.pkl', 'wb') as f:
    pickle.dump(MPdict, f)

## 저장된 파일 열어서 확인

In [219]:
filename = 'MP_news_data_4.pkl'

In [220]:
with open(filename, 'rb') as fr: 
    MPdict_r = pickle.load(fr)

In [221]:
MPdict_r.keys()

dict_keys(['강기윤', '강대식', '강득구', '강민국', '강민정', '강병원', '강선우', '강은미', '강준현', '강훈식', '고민정', '고영인', '고용진', '구자근', '권명호', '권성동', '권영세', '권은희', '권인숙', '권칠승', '기동민', '김경만', '김경협', '김교흥', '김기현', '김남국', '김도읍', '김두관', '김미애', '김민기', '김민석', '김민철', '김병기', '김병욱', '김병주', '김상훈', '김상희', '김석기', '김선교', '김성원', '김성주', '김성환', '김수흥', '김승남', '김승수', '김승원', '김영배', '김영선', '김영식', '김영주', '김영진', '김영호', '김예지', '김용민', '김용판', '김웅', '김원이', '김윤덕', '김의겸', '김정재', '김정호', '김종민', '김주영', '김진표', '김철민', '김태년', '김태호', '김학용', '김한규', '김한정', '김형동', '김홍걸', '김회재', '김희곤', '김희국', '남인순', '노용호', '노웅래', '도종환', '류성걸', '류호정', '맹성규', '문정복', '문진석', '민병덕', '민형배', '민홍철', '박광온', '박대수', '박대출', '박덕흠', '박범계', '박병석', '박상혁', '박성민', '박성준', '박성중', '박수영', '박영순', '박완주', '박용진', '박재호', '박정', '박정하', '박주민', '박진', '박찬대', '박형수', '박홍근', '배준영', '배진교', '배현진', '백종헌', '백혜련', '변재일', '서동용', '서범수', '서병수', '서삼석', '서영교', '서영석', '서일준', '서정숙', '설훈', '성일종', '소병철', '소병훈', '송갑석', '송기헌', '송석준', '송언석', '송옥주', '송재호', '신동근', '신영대', '신원식', '신정훈', '신현영', '심상정', '안규백', '안민석', '안병길',

In [222]:
MPdict_r['강기윤'].name

'강기윤'

In [226]:
MPdict_r['강기윤'].df_org.info()
MPdict_r['강기윤'].df_org.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


,title,originallink,link,description,pubDate
0,<b>강기윤</b> &quot...,http://www.edail...,https://n.news.n...,<b>강기윤</b> 국민의힘 ...,"Fri, 29 Jul 2022..."
1,<b>강기윤</b> 의원 “코...,https://mdtoday....,https://mdtoday....,▲ 국민의힘 <b>강기윤</b...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙…간사 <...,https://www.dail...,https://www.dail...,﻿﻿﻿사진설명 = 좌측부터 보...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 <b>강...,"Fri, 22 Jul 2022..."
4,&quot;팬데믹 시대 과학 ...,http://www.sisao...,http://www.sisao...,토론자: <b>강기윤</b>-...,"Tue, 26 Jul 2022..."


In [156]:
MPdict_r['강기윤'].df_r1.info()
MPdict_r['강기윤'].df_r1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         89 non-null     object
 1   originallink  89 non-null     object
 2   link          89 non-null     object
 3   description   89 non-null     object
 4   pubDate       89 non-null     object
dtypes: object(5)
memory usage: 4.2+ KB


,title,originallink,link,description,pubDate
0,확진자 접촉 무증상자도 신...,http://www.edail...,https://n.news.n...,국민의힘 의원 사진 은 이...,"Fri, 29 Jul 2022..."
1,의원 코로나 무증상자도 신...,https://mdtoday....,https://mdtoday....,국민의힘 의원 사진 의원...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙 간사 ...,https://www.dail...,https://www.dail...,사진설명 좌측부터 보건복지위...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 의원이...,"Fri, 22 Jul 2022..."
4,팬데믹 시대 과학 방역의 핵...,http://www.sisao...,http://www.sisao...,토론자 박향 정재훈 조동찬 ...,"Tue, 26 Jul 2022..."


In [157]:
type(MPdict_r['강기윤'].len), MPdict_r['강기윤'].len

(numpy.int64, 89)

In [205]:
type(MPdict_r['강기윤'].text), MPdict_r['강기윤'].text[:1000]

(str,
 '확진자 접촉 무증상자도 신속항원 검사 무료화 국민의힘 의원 사진 은 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 의원은 코로나 확진자와 접촉한 무증상자에 대해서도 의사의 판단 하에 신속항원검사비를 무료로 의원 코로나 무증상자도 신속항원검사 무료로 받을 수 있어 국민의힘 의원 사진 의원실 제공 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 등에서 의사의 판단 하에 신속항원검사 를 무료로 받게 된다 국회 보건복지위원회 소속 보건복지위원장 정춘숙 간사 강훈식 의원 사진설명 좌측부터 보건복지위원장 정춘숙 의원 국민의힘 의원 더민주당 강훈식 의원 보건복지위 위원장은 보건복지위원장 정춘숙 의원이 맡았으며 간사는 여당에서 의원 국민의힘 후반기 국회 복지위원장에 정춘숙 의원 양당 간사는 강훈식여당 간사는 국민의힘 의원이 야당 간사는 더불어민주당 강훈식 의원이 맡게됐다 의원은 전반기에 이어 후반기에도 간사 직책을 이어가게 됐으며 강훈식 의원은 전반기 팬데믹 시대 과학 방역의 핵심 관료 아닌 전문가에 맡길 것 안철수 민 당 토론자 박향 정재훈 조동찬 국민의힘 안철수 의원이 일 오전 국회 의원회관에서 열린 위기를 연합뉴스 제언 전문가 의사 결정 따른 과학방역 이뤄야 국가 책임 중요 국민의힘 의원은 코로나 재유행대비 의료체계 점검 긴급 간담회 연다 의원 국회 국민의힘 의원 경남 창원시 성산구 보건복지위원회 간사 이 오는 일 목 오후 국회의원회관 제 간담회의실에서 코로나 재유행 의료체계 점검 긴급간담회 를 개최할 예정이다 이번 간담회는 코로나 재유행대비 의료체계 점검 위한 긴급 간담회 개최국민의힘 보건복지위원회 간사인 의원 경남 창원성산구 사진 국제뉴스 국회 국민의힘 보건복지위원회 간사인 의원 창원시 성산구 에 따르면 오는 일 오후 국회의원회관 제 간담회의실에서 코로나 후반기 국회 보건복지위 정춘숙 한정애 의원 이끈다보건복지위 간사로는 국민의힘 의원과 더불어민주당 강훈식 의원이 각각 내정됐다 의원은 전반기에서도 보건복지위원회 국민의

# ★★★★발표는 여기까지★★★★★

# 계획

- 크롤링 준비
    - 국회의원이름리스트MPnames 만들기
        - 중복이 없는지 확인, 있다면 그 국회의원에 한해 신뢰도가 떨어지겠거니 일단 생각만 해둔다.
    - getNewsAbout(MPname)이름 넣으면 크롤링 데이터 반환하는 함수 만들기
        - return df_org (컬럼 구성 : ['title', 'originallink', 'link', 'description', 'pubDate'])
    - reduction1(df) 데이터 프레임을 넣으면 1차 전처리를 해서 데이터 프레임 반환하는 함수 만들기
        - return df_r1 (위와 같음)
    - reduction2(df) 데이터 프레임을 넣으면 2차 전처리를 해서 문자열 반환하는 함수 만들기
        - return text

- 크롤링
    - 만든거 for 문으로 휘리릭
    - 과정과 결과를 딕셔너리  에 저장 {MPname: [df_org, df_r1, text]}

- 1차 전처리
    - 한글이외의 것 삭제
    - 검색어 삭제
    - (title, description)으로 중복 거른다.

- 2차 전처리
    - 모든 title description을 하나의 문자열 text로 합친다.

# 준비

## import

In [131]:
import re
import pickle
from datetime import datetime
import urllib
import pandas as pd
import json

## MPnames : 국회의원 이름리스트
- MP.pkl에서 의원명 수집
- 중복이 없는지 확인, 있다면 그 국회의원에 한해 신뢰도가 떨어지겠거니 일단 생각만 해둔다.

In [132]:
MP = pd.read_pickle('MP.pkl')

In [133]:
MP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의원명     299 non-null    object
 1   정당      299 non-null    object
 2   소속위원회   298 non-null    object
 3   지역      299 non-null    object
 4   성별      299 non-null    object
 5   당선횟수    299 non-null    object
 6   당선방법    299 non-null    object
dtypes: object(7)
memory usage: 16.5+ KB


In [134]:
MPnames = MP['의원명']

In [135]:
MPnames.count(), len(MPnames.unique())

(299, 297)

### 동명이인확인
- 김병욱(33, 34), 이수진(191, 192)

In [136]:
MP[MP.duplicated(subset=['의원명'], keep='last')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
33,김병욱,더불어민주당,[국토교통위원회],경기 성남시분당구을,남,재선,지역구
191,이수진,더불어민주당,"[기획재정위원회, 예산결산특...",서울 동작구을,여,초선,지역구


In [137]:
MP[MP.duplicated(subset=['의원명'], keep='first')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
34,김병욱,국민의힘,"[교육위원회, 예산결산특별위원회]",경북 포항시남구울릉군,남,초선,지역구
192,이수진,더불어민주당,"[민생경제안정특별위원회, 국...",비례대표,여,초선,비례대표


In [138]:
for name in MPnames:
    print(name, end=" ")

강기윤 강대식 강득구 강민국 강민정 강병원 강선우 강은미 강준현 강훈식 고민정 고영인 고용진 구자근 권명호 권성동 권영세 권은희 권인숙 권칠승 기동민 김경만 김경협 김교흥 김기현 김남국 김도읍 김두관 김미애 김민기 김민석 김민철 김병기 김병욱 김병욱 김병주 김상훈 김상희 김석기 김선교 김성원 김성주 김성환 김수흥 김승남 김승수 김승원 김영배 김영선 김영식 김영주 김영진 김영호 김예지 김용민 김용판 김웅 김원이 김윤덕 김의겸 김정재 김정호 김종민 김주영 김진표 김철민 김태년 김태호 김학용 김한규 김한정 김형동 김홍걸 김회재 김희곤 김희국 남인순 노용호 노웅래 도종환 류성걸 류호정 맹성규 문정복 문진석 민병덕 민형배 민홍철 박광온 박대수 박대출 박덕흠 박범계 박병석 박상혁 박성민 박성준 박성중 박수영 박영순 박완주 박용진 박재호 박정 박정하 박주민 박진 박찬대 박형수 박홍근 배준영 배진교 배현진 백종헌 백혜련 변재일 서동용 서범수 서병수 서삼석 서영교 서영석 서일준 서정숙 설훈 성일종 소병철 소병훈 송갑석 송기헌 송석준 송언석 송옥주 송재호 신동근 신영대 신원식 신정훈 신현영 심상정 안규백 안민석 안병길 안철수 안호영 양경숙 양금희 양기대 양이원영 양정숙 양향자 어기구 엄태영 오기형 오영환 용혜인 우상호 우원식 위성곤 유경준 유기홍 유동수 유상범 유의동 유정주 윤건영 윤관석 윤두현 윤미향 윤상현 윤영덕 윤영석 윤영찬 윤재갑 윤재옥 윤주경 윤준병 윤창현 윤한홍 윤호중 윤후덕 이개호 이달곤 이동주 이만희 이명수 이병훈 이상민 이상헌 이성만 이소영 이수진 이수진 이양수 이용 이용빈 이용선 이용우 이용호 이원욱 이원택 이은주 이인선 이인영 이장섭 이재명 이재정 이정문 이종배 이종성 이주환 이채익 이철규 이탄희 이태규 이학영 이해식 이헌승 이형석 인재근 임병헌 임오경 임이자 임종성 임호선 장경태 장동혁 장제원 장철민 장혜영 전봉민 전용기 전재수 전주혜 전해철 전혜숙 정경희 정동만 정성호 정우택 정운천 정일영 정점식 정진석 정찬민 정청래 정춘숙 정태호 정필모 정희용 조경태 

## getNewsAbout(name) : 크롤링 함수
- 이름 넣으면 크롤링 데이터 반환하는 함수 만들기
- return df_org (컬럼 구성 : ['title', 'originallink', 'link', 'description', 'pubDate']) 

- https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4

In [139]:
client_id = "ISdXTGuV8Tdqx9fNxPgz"
client_secret = "e2kcexi37t"
display = 100 # 출력건수 (최대)
sort = 'sim' # 유사도

def getNewsAbout(name):    
    encText = urllib.parse.quote(name)
    url = f'https://openapi.naver.com/v1/search/news?query={encText}&display={display}&sort={sort}'

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    df = pd.DataFrame() 
    if(rescode==200):
        response_body = response.read()
    else:
        print('getNewsAbout : crawling about ', name, 'failed.')
        print("Error Code:" + rescode)
        return df # 실패시 빈 DataFrame을 return
        
    df = response_body.decode('utf-8')
    df = json.loads(df)
    df = pd.DataFrame(df['items'])
    
    return df

- 함수 기능 체크

In [140]:
name = MPnames[0]
df = getNewsAbout(name)

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [142]:
df.head()

,title,originallink,link,description,pubDate
0,<b>강기윤</b> &quot...,http://www.edail...,https://n.news.n...,<b>강기윤</b> 국민의힘 ...,"Fri, 29 Jul 2022..."
1,<b>강기윤</b> 의원 “코...,https://mdtoday....,https://mdtoday....,▲ 국민의힘 <b>강기윤</b...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙…간사 <...,https://www.dail...,https://www.dail...,﻿﻿﻿사진설명 = 좌측부터 보...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 <b>강...,"Fri, 22 Jul 2022..."
4,&quot;팬데믹 시대 과학 ...,http://www.sisao...,http://www.sisao...,토론자: <b>강기윤</b>-...,"Tue, 26 Jul 2022..."


## reduction1(df, name) : 1차 전처리 함수
- 데이터 프레임을 넣으면 1차 전처리를 해서 데이터 프레임 반환하는 함수 만들기
- return df_r1 (들어온 df와 같은 컬럼구성)

- 1차 전처리
    - (title, description)에 대하여
        - 한글이외의 것 삭제
        - 검색어 삭제
    - (title, description)을 기준으로 중복 거른다.

In [143]:
def reduction1(df, name):
    
    if df.empty:
        print("reduction1 : empty data.")
        return df
    
    df.title = df.title.apply(lambda x : re.sub('[^가-힣]+', ' ', x))
    df.description = df.description.apply(lambda x : re.sub('[^가-힣]+', ' ', x))
    
    df.title = df.title.apply(lambda x : re.sub('\s+', ' ', x))
    df.description = df.description.apply(lambda x : re.sub('\s', ' ', x))
    
    df.title = df.title.apply(lambda x : x.replace(name, ''))
    df.description = df.description.apply(lambda x : x.replace(name, ''))
    
    df.drop_duplicates(subset=['title'], inplace=True)
    df.drop_duplicates(subset=['description'], inplace=True)
    
    return df

- 함수 기능 체크

In [144]:
df1 = df.copy()
df1.info()
df_r1 = reduction1(df1, name)
df_r1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 98
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         88 non-null     object
 1   originallink  88 non-null     object
 2   link          88 non-null     object
 3   description   88 non-null     object
 4   pubDate       88 non-null     object
dtypes: object(5)
memory usage: 4.1+ KB


In [145]:
df_r1.head()

,title,originallink,link,description,pubDate
0,확진자 접촉 무증상자도 신...,http://www.edail...,https://n.news.n...,국민의힘 의원 사진 은 이...,"Fri, 29 Jul 2022..."
1,의원 코로나 무증상자도 신...,https://mdtoday....,https://mdtoday....,국민의힘 의원 사진 의원...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙 간사 ...,https://www.dail...,https://www.dail...,사진설명 좌측부터 보건복지위...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 의원이...,"Fri, 22 Jul 2022..."
4,팬데믹 시대 과학 방역의 핵...,http://www.sisao...,http://www.sisao...,토론자 박향 정재훈 조동찬 ...,"Tue, 26 Jul 2022..."


## reduction2(df) : 2차 전처리
- 데이터 프레임을 넣으면 2차 전처리를 해서 문자열 반환하는 함수 만들기
- return text

- 모든 title description을 하나의 문자열 text로 합친다.

In [146]:
def reduction2(df):
    if df.empty:
        print("reduction2 : empty data.")
        return ""
    text = ""
    sr = df.title + df.description
    text = " ".join(sr)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    
    return text

In [147]:
df_r1_ = df_r1.copy()
text = reduction2(df_r1_)
print(len(text), type(text))
print(text)

10492 <class 'str'>
확진자 접촉 무증상자도 신속항원 검사 무료화 국민의힘 의원 사진 은 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 의원은 코로나 확진자와 접촉한 무증상자에 대해서도 의사의 판단 하에 신속항원검사비를 무료로 의원 코로나 무증상자도 신속항원검사 무료로 받을 수 있어 국민의힘 의원 사진 의원실 제공 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 등에서 의사의 판단 하에 신속항원검사 를 무료로 받게 된다 국회 보건복지위원회 소속 보건복지위원장 정춘숙 간사 강훈식 의원 사진설명 좌측부터 보건복지위원장 정춘숙 의원 국민의힘 의원 더민주당 강훈식 의원 보건복지위 위원장은 보건복지위원장 정춘숙 의원이 맡았으며 간사는 여당에서 의원 국민의힘 후반기 국회 복지위원장에 정춘숙 의원 양당 간사는 강훈식여당 간사는 국민의힘 의원이 야당 간사는 더불어민주당 강훈식 의원이 맡게됐다 의원은 전반기에 이어 후반기에도 간사 직책을 이어가게 됐으며 강훈식 의원은 전반기 팬데믹 시대 과학 방역의 핵심 관료 아닌 전문가에 맡길 것 안철수 민 당 토론자 박향 정재훈 조동찬 국민의힘 안철수 의원이 일 오전 국회 의원회관에서 열린 위기를 연합뉴스 제언 전문가 의사 결정 따른 과학방역 이뤄야 국가 책임 중요 국민의힘 의원은 코로나 재유행대비 의료체계 점검 긴급 간담회 연다 의원 국회 국민의힘 의원 경남 창원시 성산구 보건복지위원회 간사 이 오는 일 목 오후 국회의원회관 제 간담회의실에서 코로나 재유행 의료체계 점검 긴급간담회 를 개최할 예정이다 이번 간담회는 코로나 재유행대비 의료체계 점검 위한 긴급 간담회 개최국민의힘 보건복지위원회 간사인 의원 경남 창원성산구 사진 국제뉴스 국회 국민의힘 보건복지위원회 간사인 의원 창원시 성산구 에 따르면 오는 일 오후 국회의원회관 제 간담회의실에서 코로나 후반기 국회 보건복지위 정춘숙 한정애 의원 이끈다보건복지위 간사로는 국민의힘 의원과 더불어민주당 강훈식 의원이 각각 내정됐다 의원은 전반기에서도

## MPdata : 데이터 저장용 클래스

In [148]:
class MPdata:
    def __init__(self, name):
        self.name = name
    def save_org(self, df_org):
        self.df_org = df_org
    def save_r1(self, df_r1):
        self.df_r1 = df_r1
        self.len = df_r1.count().max()
    def save_text(self, text):
        self.text = text

# 테스트

## 네이버 API

- https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4

In [159]:
client_id = "ISdXTGuV8Tdqx9fNxPgz"
client_secret = "e2kcexi37t"

encText = urllib.parse.quote("윤석열")
display = 100 # 출력건수 (최대)
sort = 'sim' # 유사도

In [160]:
def getDFabout(cat):
    url = f'https://openapi.naver.com/v1/search/{cat}?query={encText}&display={display}&sort={sort}'

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        print('Success')
    else:
        print("Error Code:" + rescode)
    data = response_body.decode('utf-8')
    data=json.loads(data)
    data1 = pd.DataFrame(data['items'])
    return data1

In [161]:
df = getDFabout('news')

Success


In [162]:
pd.options.display.max_colwidth = 20

In [163]:
df.columns

Index(['title', 'originallink', 'link', 'description', 'pubDate'], dtype='object')

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [206]:
df.head()

,title,originallink,link,description,pubDate
0,이재명 &quot;<b>윤석열...,https://www.yna....,https://n.news.n...,뒤 <b>윤석열</b> 정부를...,"Sun, 31 Jul 2022..."
1,[취재후 Talk] &apos...,http://news.tvch...,https://n.news.n...,지난주에 완성된 책으로 나온 ...,"Sun, 31 Jul 2022..."
2,권성동 대표 직무대행 전격 사...,https://www.khan...,https://n.news.n...,<b>윤석열</b> 정부가 출...,"Sun, 31 Jul 2022..."
3,이재명 &quot;<b>윤석열...,http://www.edail...,https://n.news.n...,더불어민주당 유력 당권 주자인...,"Sun, 31 Jul 2022..."
4,대구 찾은 이재명 &quot;...,https://www.seda...,https://n.news.n...,더불어민주당 유력 당권주자인 ...,"Sun, 31 Jul 2022..."


### description 내용 확인
- 요약이라는데 뭘 어떻게 요약한 것인가

In [166]:
df.description

0     뒤 <b>윤석열</b> 정부를...
1     지난주에 완성된 책으로 나온 ...
2     <b>윤석열</b> 정부가 출...
3     더불어민주당 유력 당권 주자인...
4     더불어민주당 유력 당권주자인 ...
             ...         
95    ☞한겨레S 뉴스레터 구독하기 ...
96    중국 정부가 27일 <b>윤석...
97    북한 김정은 위원장이 <b>윤...
98    <b>윤석열</b> 대통령이 ...
99    김정은 북한 국무위원장이 <b...
Name: description, Length: 100, dtype: object

In [167]:
description_len = df.description.apply(len)

In [168]:
description_len.describe() # 길이도.. 제각각...

count    100.000000
mean     138.090000
std       18.246403
min       83.000000
25%      131.750000
50%      139.500000
75%      149.250000
max      171.000000
Name: description, dtype: float64

In [169]:
description_len

0     170
1     150
2     131
3     137
4     146
     ... 
95    132
96    139
97    133
98    101
99    136
Name: description, Length: 100, dtype: int64

In [170]:
desc = df.description[95]

In [171]:
for d in desc:
    print(d, end="")

☞한겨레S 뉴스레터 구독하기 https://bit.ly/319DiiE 정부가 중산층과 서민의 부담을 덜어주겠다며 대대적인 감세안을 발표했다. 근로소득세, 법인세, 종합부동산세 모두 깎아주겠다고 나섰다. 근로소득세의 경우 연간 최고... 

In [172]:
desc[len(desc)-5:] # ...은 출력이 생략된게 아니라 ...자체가 문자였다.

'고... '

In [173]:
for i, desc in enumerate(df.description):
    print("*"*10, i, "*"*10)
    for d in desc:
        print(d, end="")
    print()

********** 0 **********
뒤 <b>윤석열</b> 정부를 향해 &quot;성공을 바란다&quot;고 말했다. 이 후보는 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남... 이어 &quot;나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다&quot;며 &quot;노인 일자리를 줄인다든지, 코로나 감염자... 
********** 1 **********
지난주에 완성된 책으로 나온 저의 첫 저서, &apos;청와대 출입기자가 본 조국의 시간, <b>윤석열</b>의 시간&apos; 관련... 당시 민주당 이재명 후보는 국민의힘 <b>윤석열</b> 후보에 비해 상황적으로 더 쫓기는 분위기 속에서 도리어 손발이 묶인... 
********** 2 **********
<b>윤석열</b> 정부가 출범한지 83일 만에 집권여당에서 비대위가 출범하는 초유의 사태가 발생했다. 비대위원장 임명 주체, 비대위 성격과 기간, 비대위원장 인선을 놓고 당은 더욱 격랑에 휩싸일 것으로 보인다. 권 대행은 이날... 
********** 3 **********
더불어민주당 유력 당권 주자인 이재명 당 대표 후보는 31일 대구를 찾아 <b>윤석열</b> 정부를 향해 “성공하길... 안 될 일을 한다면 싸우고 견제할 것”이라면서도 “나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다”고... 
********** 4 **********
더불어민주당 유력 당권주자인 이재명 의원이 31일 “나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다”고 말했다. 이 의원은 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남, 그리고 희망&apos;에서 &quot;상대 공격보다 국민을... 
********** 5 **********
뒤 <b>윤석열</b> 정부를 향해 &quot;성공을 바란다&quot;고 말했다. 연합뉴스에 따르면 이 후보는 이날 대구 엑스코에서 열린... 이어 &quot;나라와 국민을 위해 <b>윤석열</b> 대통령이 성

그런데 <b>윤석열</b> 대통령이 취임, <b>윤석열</b> 정부가 들어선 지 100일도 안된 시점에서 진보계 시민 단체들이 주도하는 반정부 시위가 시작됐다. 진보계 시민단체들이... 
********** 61 **********
<b>윤석열</b> 대통령이 29일 서울 용산 대통령실 청사에서 열린 주한대사 신임장 제정식에서 데시 달키 주한 에티오피아 대사로부터 신임장을 받은 뒤 차담하고 있다. (대통령실통신사진기자단) 2022.07.29. yesphoto@newsis.... 
********** 62 **********
<b>윤석열</b> 대통령에 대한 지지율이 20%대로 곤두박질친 가운데 국내에서 서서히 우려의 목소리가 나오고... [리포트] 새 정부 출범 두달만에 <b>윤석열</b> 대통령 지지율이 연일 하락세를 보이는 가운데 국내 언론에서 우려의... 
********** 63 **********
7월26일 권성동 국민의힘 당대표 직무대행 겸 원내대표와 <b>윤석열</b> 대통령 간 텔레그램 메시지가 포착됐습니다.... 저는 <b>윤석열</b> 대통령도 사람인데 얼마든지 이준석 대표에 대한 감정을 사적으로 대화하는 게 뭐가 문제냐.... 
********** 64 **********
<b>윤석열</b> 대통령의 배우자 김건희 여사 팬클럽인 &apos;건희사랑&apos; 회장을 맡았던 강신업 변호사가 28일 자리에서... 김건희 여사와 <b>윤석열</b> 대통령을 지켜야 하지만 오히려 물러남으로서 더 큰 것을 얻을 수 있다고 생각한다&quot;며... 
********** 65 **********
원내대표와 <b>윤석열</b> 대통령 간 텔레그램 메시지가 포착됐습니다. 윤 대통령이 &quot;우리당도 잘하네요 계속... ▲  채널A &lt;뉴스TOP10&gt;(7월 27일) 대담 일부 ⓒ 민주언론시민연합 ▲  <b>윤석열</b> 대통령 ‘내부 총질’ 표현을 정조... 
********** 66 **********
김정은 북한 국무위원장이 28일 <b>윤

In [174]:
df.description[0]

'뒤 <b>윤석열</b> 정부를 향해 &quot;성공을 바란다&quot;고 말했다. 이 후보는 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남... 이어 &quot;나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다&quot;며 &quot;노인 일자리를 줄인다든지, 코로나 감염자... '

In [175]:
print(df.link[0])

https://n.news.naver.com/mnews/article/001/0013345947?sid=100


In [176]:
# 본문에서 검색어의 단어가 포함된 문장(+뒷 문장 일부)을 가져온다. 2번째 일치 되는 것까지만..

## 전처리

- 1차
    - 한글이외의 것 삭제
    - 검색어 삭제 (여기선 윤석열)
    - title, description으로 중복 거른다.

- 2차
    - 모든 title description을 하나의 문자열 text로 합친다.

In [177]:
df.drop_duplicates(subset=['title']).drop_duplicates(subset=['description']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         84 non-null     object
 1   originallink  84 non-null     object
 2   link          84 non-null     object
 3   description   84 non-null     object
 4   pubDate       84 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [179]:
sr = df.title + df.description
sr.info(), sr[0], type(sr), type(sr[0])

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes


(None,
 '이재명 &quot;<b>윤석열</b> 대통령 성공하길…민주당은 바뀌어야&quot;뒤 <b>윤석열</b> 정부를 향해 &quot;성공을 바란다&quot;고 말했다. 이 후보는 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남... 이어 &quot;나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다&quot;며 &quot;노인 일자리를 줄인다든지, 코로나 감염자... ',
 pandas.core.series.Series,
 str)

In [180]:
sr

0     이재명 &quot;<b>윤석열...
1     [취재후 Talk] &apos...
2     권성동 대표 직무대행 전격 사...
3     이재명 &quot;<b>윤석열...
4     대구 찾은 이재명 &quot;...
             ...         
95    연봉 3천은 혜택 0원…<b>...
96    中, <b>윤석열</b> 정부...
97    김정은, 尹 이름 걸고 첫 비...
98    주한 몽골대사 신임장 받는 <...
99    김정은 “선제타격 시도땐 <b...
Length: 100, dtype: object

In [181]:
" ".join(sr)

'이재명 &quot;<b>윤석열</b> 대통령 성공하길…민주당은 바뀌어야&quot;뒤 <b>윤석열</b> 정부를 향해 &quot;성공을 바란다&quot;고 말했다. 이 후보는 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남... 이어 &quot;나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다&quot;며 &quot;노인 일자리를 줄인다든지, 코로나 감염자...  [취재후 Talk] &apos;청와대 출입기자가 본 조국의 시간, <b>윤석열</b>의 시간&apos;지난주에 완성된 책으로 나온 저의 첫 저서, &apos;청와대 출입기자가 본 조국의 시간, <b>윤석열</b>의 시간&apos; 관련... 당시 민주당 이재명 후보는 국민의힘 <b>윤석열</b> 후보에 비해 상황적으로 더 쫓기는 분위기 속에서 도리어 손발이 묶인...  권성동 대표 직무대행 전격 사퇴···<b>윤석열</b> 정부 출범 83일만에 비대위 전환...<b>윤석열</b> 정부가 출범한지 83일 만에 집권여당에서 비대위가 출범하는 초유의 사태가 발생했다. 비대위원장 임명 주체, 비대위 성격과 기간, 비대위원장 인선을 놓고 당은 더욱 격랑에 휩싸일 것으로 보인다. 권 대행은 이날...  이재명 &quot;<b>윤석열</b>, 성공하기 바라…안 될 일 하며 싸울 것&quot;더불어민주당 유력 당권 주자인 이재명 당 대표 후보는 31일 대구를 찾아 <b>윤석열</b> 정부를 향해 “성공하길... 안 될 일을 한다면 싸우고 견제할 것”이라면서도 “나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다”고...  대구 찾은 이재명 &quot;<b>윤석열</b> 성공하길…민주당은 바뀌어야&quot;더불어민주당 유력 당권주자인 이재명 의원이 31일 “나라와 국민을 위해 <b>윤석열</b> 대통령이 성공하길 바란다”고 말했다. 이 의원은 이날 대구 엑스코에서 열린 시민 토크쇼 &apos;만남, 그리고 희망&apos;에서 &quot;상대 공격보다 국민을...  &apos;보수의 

# (시간되면)

## text로 단어의 리스트 만들기
- split하면 됨

## wordWhere = {word :  list_of_index} 이런 딕셔너리 만들기

In [182]:
text = '강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'

In [183]:
words = text.split()

In [184]:
words

['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지', '강쥐']

In [185]:
import numpy as np

In [186]:
words = np.array(words)
words

array(['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지',
       '강쥐'], dtype='<U4')

In [187]:
np.where(words == '강아지')

(array([0, 3, 5, 9], dtype=int64),)

In [188]:
words

array(['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지',
       '강쥐'], dtype='<U4')

## 형태소 분석과 조사 삭제

In [189]:
from konlpy.tag import Okt

In [190]:
text = '강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'
text

'강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'

In [191]:
okt = Okt()
morph = okt.pos(text)

In [192]:
morph

[('강아지', 'Noun'),
 ('으르릉', 'Noun'),
 ('멍멍', 'Adverb'),
 ('강아지', 'Noun'),
 ('악', 'Noun'),
 ('강아지', 'Noun'),
 ('강', 'Noun'),
 ('아', 'Josa'),
 ('아앙', 'Noun'),
 ('강', 'Noun'),
 ('아', 'Josa'),
 ('쥐', 'Noun'),
 ('갱얼쥐', 'Noun'),
 ('강아지', 'Noun'),
 ('강쥐', 'Noun')]

- NOTE
    - 의미를 모르고 쪼개는 것 같다. 무슨 뜻의 단어로 보고 분류했는지 알기 어렵다.
    - 한국어는 같은 단어에 전혀 다른 뜻이 쓰이는 경우가 많아서 어려울듯.
        - ex) 강 river strong/ 말 hourse words/ 눈 snow, eyes
    - 의미 분석을 하려면 영어로 번역해서 하는 것도 방법일듯
    - 네이버 파파고도 API 있다.

- Okt.
    - jki morphs nomalize nouns phrases pos tagset

In [193]:
okt.jki

<java object 'kr.lucypark.okt.OktInterface'>

In [194]:
okt.morphs(text)

['강아지',
 '으르릉',
 '멍멍',
 '강아지',
 '악',
 '강아지',
 '강',
 '아',
 '아앙',
 '강',
 '아',
 '쥐',
 '갱얼쥐',
 '강아지',
 '강쥐']

In [195]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [196]:
okt.pos('코로나19')

[('코로나', 'Noun'), ('19', 'Number')]

In [197]:
okt.pos('이놈아')

[('이', 'Determiner'), ('놈', 'Noun'), ('아', 'Josa')]

In [198]:
okt.pos('완전 짜증 났잖아~~ 아오 진짜 내말이 그말임')

[('완전', 'Noun'),
 ('짜증', 'Noun'),
 ('났잖아', 'Verb'),
 ('~~', 'Punctuation'),
 ('아오', 'Exclamation'),
 ('진짜', 'Noun'),
 ('내', 'Determiner'),
 ('말', 'Noun'),
 ('이', 'Josa'),
 ('그', 'Determiner'),
 ('말임', 'Noun')]

In [199]:
sample_text = "정부 새로운 인수위 조직 중 코로나비상대응특위 위원으로 인선됐다고 이준석에 읍소하는 국민의힘 지방선거 창원시장 후보 공천 컷오프에 반발해 재심을 신청한 창원 성산 가운데 국민의힘 지방선거 창원시장 후보 공천 컷오프에 반발해 재심을 신청한 창원 성산 가운데 의원 대통령직 인수위원회 합류 코로나비상대응특위 위원으로 의원 경남 창원성산구 보건복지위원회 간사 은 지난 일 윤석열 정부 새로운 인수위 조직 중 현역의원인 의원도 인수위원회에 합류하게 되었다 사진 출처 의원 트위터 강 의원실 의원 정부 코로나비상대응특위 위원으로 인선 의원 사진 의원실 제공 국민의힘 의원이 윤석열 정부의 코로나 비생대응 특별위원회의 위원으로 선정됐다 국회 보건복지위원회 간사 국민의힘 의원은 윤석열 정부 새로운 인수위 방역 붕괴 속 다양한 치료제 준비 미흡 질타 의원 경남 창원성산구 보건복지위원회 간사 이 방역 붕괴 속에도 정부가 소트로비맙 몰누피라비르 등 다양한 치료제 준비가 미흡하고 구체적인 검토조차 하지 않았다 고 지적했다 의원은 일 정부가 인수위 코로나비상대응특위 방역현장 의견청취해"
sample_text

'정부 새로운 인수위 조직 중 코로나비상대응특위 위원으로 인선됐다고 이준석에 읍소하는 국민의힘 지방선거 창원시장 후보 공천 컷오프에 반발해 재심을 신청한 창원 성산 가운데 국민의힘 지방선거 창원시장 후보 공천 컷오프에 반발해 재심을 신청한 창원 성산 가운데 의원 대통령직 인수위원회 합류 코로나비상대응특위 위원으로 의원 경남 창원성산구 보건복지위원회 간사 은 지난 일 윤석열 정부 새로운 인수위 조직 중 현역의원인 의원도 인수위원회에 합류하게 되었다 사진 출처 의원 트위터 강 의원실 의원 정부 코로나비상대응특위 위원으로 인선 의원 사진 의원실 제공 국민의힘 의원이 윤석열 정부의 코로나 비생대응 특별위원회의 위원으로 선정됐다 국회 보건복지위원회 간사 국민의힘 의원은 윤석열 정부 새로운 인수위 방역 붕괴 속 다양한 치료제 준비 미흡 질타 의원 경남 창원성산구 보건복지위원회 간사 이 방역 붕괴 속에도 정부가 소트로비맙 몰누피라비르 등 다양한 치료제 준비가 미흡하고 구체적인 검토조차 하지 않았다 고 지적했다 의원은 일 정부가 인수위 코로나비상대응특위 방역현장 의견청취해'

In [200]:
okt.pos(sample_text)

[('정부', 'Noun'),
 ('새로운', 'Adjective'),
 ('인수위', 'Noun'),
 ('조직', 'Noun'),
 ('중', 'Noun'),
 ('코로나비', 'Noun'),
 ('상', 'Suffix'),
 ('대응', 'Noun'),
 ('특위', 'Noun'),
 ('위원', 'Noun'),
 ('으로', 'Josa'),
 ('인선', 'Noun'),
 ('됐다고', 'Verb'),
 ('이준석', 'Noun'),
 ('에', 'Josa'),
 ('읍', 'Noun'),
 ('소하', 'Noun'),
 ('는', 'Josa'),
 ('국민', 'Noun'),
 ('의', 'Josa'),
 ('힘', 'Noun'),
 ('지방선거', 'Noun'),
 ('창원시', 'Noun'),
 ('장', 'Suffix'),
 ('후보', 'Noun'),
 ('공천', 'Noun'),
 ('컷오프', 'Noun'),
 ('에', 'Josa'),
 ('반발', 'Noun'),
 ('해', 'Verb'),
 ('재심', 'Noun'),
 ('을', 'Josa'),
 ('신청', 'Noun'),
 ('한', 'Josa'),
 ('창원', 'Noun'),
 ('성산', 'Noun'),
 ('가운데', 'Noun'),
 ('국민', 'Noun'),
 ('의', 'Josa'),
 ('힘', 'Noun'),
 ('지방선거', 'Noun'),
 ('창원시', 'Noun'),
 ('장', 'Suffix'),
 ('후보', 'Noun'),
 ('공천', 'Noun'),
 ('컷오프', 'Noun'),
 ('에', 'Josa'),
 ('반발', 'Noun'),
 ('해', 'Verb'),
 ('재심', 'Noun'),
 ('을', 'Josa'),
 ('신청', 'Noun'),
 ('한', 'Josa'),
 ('창원', 'Noun'),
 ('성산', 'Noun'),
 ('가운데', 'Noun'),
 ('의원', 'Noun'),
 ('대통령직', 'Noun'),
 ('인

## 빈도 확인
## 불용어 리스트 만들기
## 막대그래프랑 워드클라우드 살짝 뽑아보기